In [ ]:

# coding: utf-8

# In[6]:

import os
import argparse
import logging
logging.basicConfig(level=logging.DEBUG)
from common import find_mxnet, data, fit
from common.util import download_file
from tf_iterators import *
from ilab_iterator import ilab_iterator, Multi_ilab_iterator, Single_ilab_iterator, random_task_iterator
import mxnet as mx
import numpy as np
import numpy 
from mxnet import nd

class Cross_Entropy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self):
        super(Cross_Entropy, self).__init__('cross-entropy')
    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)
        label = labels[0].asnumpy()
        pred = preds[0].asnumpy()
        for i in range(label.shape[0]):
            prob = pred[i,numpy.int64(label[i])]
            if len(labels) == 1:
                self.sum_metric += (-numpy.log(prob)).sum()
        self.num_inst += label.shape[0]


# In[7]:


# In[8]:

batch_size = 256
data_dir_hum='/efs/datasets/users/furlat/human_sketches'
data_dir_sketchy='/efs/datasets/users/furlat/sketchy_database'
train_sketches, val_sketches = sketchy_iterator(data_dir_sketchy,batch_size)
train_sketches, val_sketches = human_sketches_iterator(data_dir_hum,batch_size)

image_shape = '3,224,224'
label_names = [train_sketches.provide_label[0][0]]
num_classes=[365]
# ctx=[mx.gpu(0),mx.gpu(1),mx.gpu(2),mx.gpu(3),mx.gpu(4),mx.gpu(5),mx.gpu(6),mx.gpu(7),mx.gpu(8),mx.gpu(9),mx.gpu(10),mx.gpu(11),mx.gpu(12),mx.gpu(13),mx.gpu(14),mx.gpu(15)]
ctx=[mx.gpu(0),mx.gpu(1),mx.gpu(2),mx.gpu(3),mx.gpu(4),mx.gpu(5)]


# In[9]:

from importlib import import_module
net = import_module('symbols.resnet_factory')
sym,data,labels=net.get_symbol_2branch(num_classes=250,active=1, gate_prefix=None,rescale_grad=1, num_layers=50,gated=True, image_shape=image_shape,coupled=True)


# In[10]:



# In[9]:




In [ ]:
#mx.viz.plot_network(symbol=sym)
imagenet_weights= '/efs/datasets/users/furlat/results_imagenet/imagenet_r50-lr05'
#model_prefix= 'tryhard-resnet'
prefix = imagenet_weights
#prefix = model_prefix
epoch=120
save_dict = nd.load('%s-%04d.params' % (prefix, epoch))
arg_params_imag = {}
aux_params_imag = {}
ext_check=['sc','fc1','data']
imagenet_par=[]
exact_check=['bn1_beta','bn1_gamma']
for k, v in save_dict.items():
    tp, name = k.split(':', 1)

    if tp == 'arg':
        arg_params_imag[name] = v
        
        
        #print name
        if not any(ext in name for ext in ext_check):
            if not any(ext == name for ext in exact_check):
                imagenet_par.append(name)
                arg_params_imag['_a_'+name] = v


    if tp == 'aux':
        aux_params_imag[name] = v
        aux_params_imag['_a_'+name] = v
del arg_params_imag['fc1_bias']
del arg_params_imag['fc1_weight']



#arg_params_imag.list_arguments


#_a_bn_data_beta

symlist=sym.list_arguments()
gatelist=[s for s in symlist if 'gate' in s]

In [ ]:
#imagenet_par

In [ ]:



# In[ ]:

mod = mx.mod.Module(sym, label_names=label_names,fixed_param_names=imagenet_par,context=ctx)
#
#checkpoint = mx.callback.module_checkpoint(mod,model_prefix)
mod.bind(data_shapes=train_sketches.provide_data, label_shapes=val_sketches.provide_label)
mod.init_params(initializer=mx.initializer.Uniform(0.01), arg_params=arg_params_imag, aux_params=aux_params_imag,
                    allow_missing=True, force_init=False)
model_prefix = '/home/ubuntu/results/resnet-2streams-testingthisgateyo-sketchy' 
checkpoint = mx.callback.module_checkpoint(mod,model_prefix)

# for gate in gatelist:
#     print mod.get_params()[0][gate].asnumpy(), gate

# In[7]:




In [ ]:
mod.fit(train_sketches,
         eval_data=val_sketches,
         eval_metric=[Cross_Entropy()],
         #eval_metric=[mx.metric.Accuracy()],

         batch_end_callback = [mx.callback.log_train_metric(1),mx.callback.Speedometer(batch_size,1)],
         epoch_end_callback=checkpoint,
         allow_missing=False,
         begin_epoch=0,
         log_prefix = model_prefix,
         optimizer_params={'learning_rate':0.1, 'momentum': 0.9,'wd':0.0004 },
         num_epoch=40)

In [ ]:
# for gate in gatelist:
#         print mod.get_params()[0][gate].asnumpy(), gate
# res_train = mod.score(train_sketches, mx.metric.Accuracy(),num_batch=40)
# res_val= mod.score(val_sketches, mx.metric.Accuracy(),num_batch=40)        

In [ ]:
# i=1
# for name, value in res_train:
#         print 'Epoch[%d] Training-%s=%f' %(i+1, name, value)
# for name, value in res_val:
#         print 'Epoch[%d] Validation-%s=%f' %(i+1, name, value)


In [ ]:


# In[ ]:
# 
train_score=[]
val_score=[]
epoch=[]
num_batch=
for i in range(40):
    
    sym, arg_params, aux_params =             mx.model.load_checkpoint(model_prefix, i+1)
        
    mod.set_params(arg_params, aux_params)
    
    for gate in gatelist:
        print mod.get_params()[0][gate].asnumpy(), gate
    res_train = mod.score(train_sketches, mx.metric.Accuracy(),num_batch=40)
    res_val= mod.score(val_sketches, mx.metric.Accuracy(),num_batch=40)
    epoch.append(i+1)
    for name, value in res_train:
        print 'Epoch[%d] Training-%s=%f' %(i+1, name, value)
        train_score.append(value)
    for name, value in res_val:
        print 'Epoch[%d] Validation-%s=%f' %(i+1, name, value)
        val_score.append(value)
        
for gate in gatelist:
    print mod.get_params()[0][gate].asnumpy(), gate


# In[ ]:

logfile_url= '%s-eval-metric-log-accuracy.txt' % (model_prefix)
print 'saving logfiles  at %s' % (logfile_url)
logfile = open(logfile_url, 'a')
for  epoch,train_metric, val_metric in zip(epoch,train_score,val_score):
    #logfile.write("%s\n" % item)


    logfile.write(str(epoch)+"\t"+str(train_metric)+"\t"+ str(val_metric)+"\n")
logfile.close()

